## Dataset Tráfico Históricos

En este documento procesamos los datos históricos de tráfico (años 2015, 2016 y 2017(meses 01-02 y 03).
Los pasos que seguimos son los siguientes:

1) Crear una base de datos Mongo para almacenar los datos una vez procesados y enriquecidos.

2) Cargar las coordenadas de cada estación (fichero __pmed_trafico_coord.csv__ proporcionado por Borja)

3) Procesar de forma separada cada año

* Los años 2015 y 2016 mantienen un formato común. Cada mes es un fichero CSV  comprimido con datos separados por ';'.
* Los meses de Enero y Febrero de 2017 mantienen el mismo formato que 2015 y 2016. __El mes 03 de 2017 es un CSV comprimido con dato separados por ','__.
* Fechas en formato datetime
* Coordenadas en formato (x,y)
* Eliminadas las medidiones de estaciones sin coordenadas

4) Una vez procesados los años se almacenan en Mongo. 
    
* La base de datos se llama *trafico*
* Tres colecciones
    * trafico2015 (111257705 documentos , 3 horas de proceso)
    * trafico2016 (119050059 documentos , 3 horas de proceso)
    * trafico2017 ( 28789567 documentos)

In [ ]:
import pymongo 
import pandas as pd
import numpy as np
import json
import os
from os import listdir
from os.path import isfile, join
from pandas.io.json import json_normalize
import zipfile
import time
# Mostrar la hora en formato HORAS:MINUTOS:SEGUNDOS
print (time.strftime('%H:%M:%S'))
import re
from dateutil.parser import *

### Mongo

In [ ]:
# MONGO
def open_conection():
    client = pymongo.MongoClient('localhost',27017)
    return client

def storeTrafico_ant(col, tabla):
    d = tabla.to_json(orient='records', date_format = 'iso', date_unit = 's')
    data_json = json.loads(d)
    col.insert_many(data_json)   
    
def storeTrafico(col, tabla):
    d = tabla.to_json(orient='records', date_format = 'iso')
    data_json = json.loads(d)
    #print(data_json[0])
    for d in data_json:
        d['fecha']= parse(d['fecha'])
    col.insert_many(data_json) 

In [ ]:
# borrar la base de datos tráfico y crearse una nueva. Solo si no existe ya la BD trafico
# Esto solo lo ejecuto una vez en la vida
def crearDBTraffic(client):
    client.drop_database('trafico_timestamp')
    return True

"""
client = open_conection()
crearDBTraffic(client)
client.close()
"""

In [ ]:
client = open_conection()
db = client.trafico_timestamp

In [ ]:
client

## Definición de funciones auxiliares

En cuanto al enriquecimiento, elimino todas aquellas entradas que corresponden con estaciones sin coordenadas.

### Rutas 

In [ ]:
historico_2015 = "./TRAFICO/historico/2015-20170427T093654Z-001/2015"
historico_2016 = "./TRAFICO/historico/2016"
historico_2017 = "./TRAFICO/historico/2017"

### Tráfico

In [ ]:
def extract_files(path):
    """
    input: directorio donde están los ficheros zip de un año. Uno por  mes
    output: devuelve la lista de los ficheros .zip de cada mes
    """
    return [ path +'/' + f for f in listdir(path) ]

def df_month(file):
    """ 
    input: fichero en formato csv
    output: dataframe     """
    return pd.read_csv(file,  sep = ';', parse_dates =[1])

def crear_csv(filemes, tabla):
    tabla.to_csv(filemes, index = False)

def df_resample(data):
    fagg = { 'identif': min, 'tipo_elem': min, 'intensidad': np.mean,
             'ocupacion': np.mean, 'carga': np.mean, 'vmed': np.mean,
             'error': min, 'periodo_integracion' : np.mean}
    
    return data.resample('H', on='fecha').apply(fagg).dropna()

def agghora(dfmonth):
    return dfmonth.groupby(by =['idelem']).apply(df_resample)



def punto_medida(x):
    m30 = 'M-30'
    urbanos = 'URBANOS'
    res = re.search(m30, str(x))
    if res:
        return res.group(0)
    else: 
        res = re.search(urbanos, str(x))
        if res:
            return res.group(0)
        else: 
            return x
    
def process_year(zipfiles, coord, dbcol, modo = 'csv'):    
    # creamos tantos dataframes como ficheros csv. Una tabla por mes
    for filezip in zipfiles:     # 01-2015.zip
        zf = zipfile.ZipFile(filezip, 'r')
        csvfile = zf.namelist()[0]
        month = df_month(zf.open(csvfile))  # dataframe de un año 
        month = agghora(month)              # agrego los datos por hora (vienen cada 15 minutos)
        month.reset_index(inplace = True)
        month.fecha = month.fecha + pd.to_timedelta(1, unit = 'h')
        month.tipo_elem = month.tipo_elem.apply(lambda x :punto_medida(x))
        month_coor = pd.merge(month, coord,  on = 'idelem' ) # con coordenadas
        if modo != 'csv':
        # llevarlo a json
            storeTrafico(dbcol, month_coor)        
        else: 
            # llevarlo a csv
            crear_csv("./procesados/" + csvfile, month_coor)
        print('Mes: ', csvfile)
    return True

#### Comprobar el árbol de ficheros


In [ ]:
## 2015
#---------------------------------------------------------------------------
# proceso de validación de los ficheros
# comprobar que dentro de cada ZIP haya un fichero csv con el mismo nombre
#---------------------------------------------------------------------------
monthZIPfiles_2015 = extract_files(historico_2015)
for filezip in monthZIPfiles_2015:     # 01-2015.zip
    zf = zipfile.ZipFile(filezip, 'r')
    print(zf.namelist())

In [ ]:
## 2016
#---------------------------------------------------------------------------
# proceso de validación de los ficheros
# comprobar que dentro de cada ZIP haya un fichero csv con el mismo nombre
#---------------------------------------------------------------------------
monthZIPfiles_2016 = extract_files(historico_2016)
for filezip in monthZIPfiles_2016:     # 01-2015.zip
    zf = zipfile.ZipFile(filezip, 'r')
    print(zf.namelist())

In [ ]:
## 2017 
#---------------------------------------------------------------------------
# proceso de validación de los ficheros
# comprobar que dentro de cada ZIP haya un fichero csv con el mismo nombre
#---------------------------------------------------------------------------
monthZIPfiles_2017 = extract_files(historico_2017)
for filezip in monthZIPfiles_2017:     # 01-2015.zip
    zf = zipfile.ZipFile(filezip, 'r')
    print(zf.namelist())
    
# Solo tenemos los datos hasta abril
# Ojo: el mes de marzo viene separado por , en lugar de ;

## main()
Crear los csvs de todos los meses de cada año

In [ ]:
if client:
    print('Cierro conexion mongo')
    client.close()

In [ ]:
# Abrir conexión mongo y accedemos a la bd Trafico
client = open_conection()
db = client.trafico_timestamp

In [ ]:
# leemos el fichero de las coordenadas
coordenadas = pd.read_csv('./pmed_trafico_coord.csv', sep = ';', decimal= b',',
                         usecols = ['idelem', 'cod_cent', 'Xcoord', 'Ycoord'])


### Año 2015

In [ ]:
## 2015 Ejecución de todo el 2015
print (time.strftime('%H:%M:%S'))
col2015 = db.trafico2015
monthZIPfiles_2015 = extract_files(historico_2015)
b2015 = process_year(monthZIPfiles_2015, coordenadas, col2015, modo = 'csv')
print (time.strftime('%H:%M:%S'))
b2015

In [ ]:
col2015.count()    #número de json cargados en mongo

### Año 2016

In [ ]:
## 2016 Ejecución de todo el 2016
col2016 = db.trafico2016
monthZIPfiles_2016 = extract_files(historico_2016)
b2016 = process_year(monthZIPfiles_2016, coordenadas, col2016, modo = 'csv')
b2016

In [ ]:
col2016.count()  #número de json cargados en mongo

### Año 2017 Enero y Febrero

In [ ]:
## 2017 Ejecución de enero y febrero de 2017
print (time.strftime('%H:%M:%S'))
col2017 = db.trafico2017
monthZIPfiles_2017 = extract_files(historico_2017)
# proceso los dos primeros meses:
b2017 = process_year(monthZIPfiles_2017[:2], coordenadas, col2017, modo = 'csv')

print (time.strftime('%H:%M:%S'))

### Año 2017 Marzo

In [ ]:
# Ahora proceso el mes 3 que viene separado por ,
print (time.strftime('%H:%M:%S'))
col2017 = db.trafico20178
col2017

In [ ]:
zf = zipfile.ZipFile(monthZIPfiles_2017[2], 'r')
csvfile = zf.namelist()[0]
month = pd.read_csv(zf.open(csvfile), sep =',', parse_dates =[1])

In [ ]:
month = agghora(month)

In [ ]:
month.reset_index(inplace = True)

In [ ]:
month.tipo_elem = month.tipo_elem.apply(lambda x :punto_medida(x))
month_coor = pd.merge(month, coordenadas,  on = 'idelem' ) # con coordenadas

In [ ]:
# almacenar en mongo
storeTrafico(col2017, month_coor)

In [ ]:
# almacenar en csv
crear_csv("./procesados/" + csvfile, month_coor)

In [ ]:
client.close()

## Leer csvs y guardar en mongo

In [ ]:
client = open_conection()
db = client.trafico
client, db

In [ ]:
ruta_trafico_procesados = "./procesados"
files = extract_files(ruta_trafico_procesados)
files

In [ ]:
col = db.trafico2015
col.drop() # borro la colección entera
for file in files:   
    # vuelvo a mongo los de de 2017 
    if  int(file[-8:-4]) != 2015 :
        continue
    col = db.trafico2015
    df_traf = pd.read_csv(file, parse_dates = [1], encoding='latin-1')   
    df_traf.fecha = df_traf.fecha.map(lambda x: pd.Timestamp(x))
    storeTrafico(col, df_traf)

In [ ]:
client.close()

In [ ]:
def count_tuples(files):
    month_files = files
    cont = 0
    for mesxlsx in month_files:
        # juego solo con los de 2017 (hecho)
        if  int(mesxlsx[-8:-4]) != 2017 :
            continue
        #juego solo con los de 2016
        #if  int(mesxlsx[-8:-4]) != 2016 :
        #    continue
        # juego solo con los de 2015
        """        if  int(mesxlsx[-8:-4]) != 2015 :
            continue"""
        
        month = pd.read_csv(mesxlsx,  encoding='latin-1', 
                              parse_dates = [4])
        print(mesxlsx, len(month))
        cont = cont + len(month) 
    print(cont)